In [ ]:
import pandas as pd
import numpy as np


# Importando as bases

df_doc_1 = pd.read_excel('C:\\Users\\scosta\\Desktop\\Faturamento.xlsx') # Histórico de Faturamento 1
df_doc_2 = pd.read_excel('C:\\Users\\scosta\\Desktop\\Faturamento_2.xlsx') # Histórico de Faturamento 2
df_05 = pd.read_excel('C:\\Users\\scosta\\Desktop\\Análise Disponibilidade.xlsx', sheet_name='VA05_Hoje') # Análise de Disponibilidades dos Pedidos Pendentes de Atendimento

print("Tabelas importadas com sucesso")



df_doc_1 - Faturamento

df_doc_2 - Separação




## Faturamento ##


In [ ]:
# Filtrando a df_doc_1 todos os itens faturados
indices_a_remover = df_doc_1[df_doc_1['TpDocVend.'] != 'Z000'].index
df_doc_1 = df_doc_1.drop(indices_a_remover)
indices_a_remover_2 = df_doc_1[df_doc_1['G~Ctg.NF'] != '1N'].index
df_doc_1 = df_doc_1.drop(indices_a_remover_2)
indices_a_remover_3 = df_doc_1[df_doc_1['G~Estornado'] == 'X'].index
df_doc_1 = df_doc_1.drop(indices_a_remover_3)

# Transformando colunas de datas
df_doc_1['F~Dt.fatur.'] = pd.to_datetime(df_doc_1['F~Dt.fatur.'])
df_doc_1['A~DtRefClnt.'] = pd.to_datetime(df_doc_1['A~DtRefClnt.'])

linhas = len(df_doc_1)
print(linhas)

## Filtrando apenas o mês de interesse do faturamento ##

In [ ]:
# Filtrando apenas o mês de interesse
data_inicio_faturamento = pd.Timestamp('2024-12-01')
data_fim_faturamento = pd.Timestamp('2024-12-31')
df_doc_1 = df_doc_1[(df_doc_1['F~Dt.fatur.'] >= data_inicio_faturamento) & (df_doc_1['F~Dt.fatur.'] <= data_fim_faturamento)]

linhas_2 = len(df_doc_1)
print(linhas_2)

In [ ]:
# Inserindo coluna com a diferença entre as datas de faturamento e datas do pedido do cliente
df_doc_1['Diferenca_Dias'] = (df_doc_1['F~Dt.fatur.'] - df_doc_1['A~DtRefClnt.']).dt.days

# Exibir as primeiras linhas do DataFrame para verificar a nova coluna
print(df_doc_1[['F~Dt.fatur.', 'A~DtRefClnt.', 'Diferenca_Dias']].head())

In [ ]:
# Inserindo coluna com o nome do tipo de pedido

# Dicionário para mapear os valores de 'A~MotivoOrd' para os tipos de pedido
mapeamento_tipo_pedido = {
    'Z20': 'Express',
    'Z25': 'Semanal',
    'Z22': 'Estoque',
    'Z35': 'Programado'
}

# Criar a coluna 'Tipo_Pedido' usando o mapeamento
df_doc_1['Tipo_Pedido'] = df_doc_1['A~MotivoOrd'].map(mapeamento_tipo_pedido)

#df_doc_1.head()

Quantidade de dia para cada tipo de pedido:


Express - 4 dias
Semanal - 7 dias
Estoque - 21 dias
Programado - 120 dias

In [ ]:
# Inserindo coluna com a verificação se o pedido faturado no mês de interesse atendeu os dias especificados

# Definir as condições de acordo com os valores de 'A~MotivoOrd' e 'Diferenca_Dias'
condicoes = [
    ((df_doc_1['A~MotivoOrd'] == 'Z20') & (df_doc_1['Diferenca_Dias'] >= 0) & (df_doc_1['Diferenca_Dias'] <= 4)),
    ((df_doc_1['A~MotivoOrd'] == 'Z25') & (df_doc_1['Diferenca_Dias'] >= 0) & (df_doc_1['Diferenca_Dias'] <= 7)),
    ((df_doc_1['A~MotivoOrd'] == 'Z22') & (df_doc_1['Diferenca_Dias'] >= 0) & (df_doc_1['Diferenca_Dias'] <= 21)),
    ((df_doc_1['A~MotivoOrd'] == 'Z35') & (df_doc_1['Diferenca_Dias'] >= 0) & (df_doc_1['Diferenca_Dias'] <= 120))
]

# Definir as escolhas correspondentes a cada condição
escolhas = [
    'Atendeu no Prazo',
    'Atendeu no Prazo',
    'Atendeu no Prazo',
    'Atendeu no Prazo'
]

# Aplicar as condições e escolhas no DataFrame, 'Não Atendeu' é o padrão para quando nenhuma condição é verdadeira
df_doc_1['Atendimento'] = np.select(condicoes, escolhas, default='Não Atendeu')

#df_doc_1

In [ ]:
# Contagem de linhas por tipo de pedido - Faturados

# Verifica ambas as condições desejadas
condicao_1 = (df_doc_1['Tipo_Pedido'] == 'Express') & (df_doc_1['Atendimento'] == 'Atendeu no Prazo')
condicao_2 = (df_doc_1['Tipo_Pedido'] == 'Express') & (df_doc_1['Atendimento'] == 'Não Atendeu')

condicao_3 = (df_doc_1['Tipo_Pedido'] == 'Semanal') & (df_doc_1['Atendimento'] == 'Atendeu no Prazo')
condicao_4 = (df_doc_1['Tipo_Pedido'] == 'Semanal') & (df_doc_1['Atendimento'] == 'Não Atendeu')

condicao_5 = (df_doc_1['Tipo_Pedido'] == 'Estoque') & (df_doc_1['Atendimento'] == 'Atendeu no Prazo')
condicao_6 = (df_doc_1['Tipo_Pedido'] == 'Estoque') & (df_doc_1['Atendimento'] == 'Não Atendeu')

condicao_7 = (df_doc_1['Tipo_Pedido'] == 'Programado') & (df_doc_1['Atendimento'] == 'Atendeu no Prazo')
condicao_8 = (df_doc_1['Tipo_Pedido'] == 'Programado') & (df_doc_1['Atendimento'] == 'Não Atendeu')


# Contar as linhas que satisfazem a condição
express_s1 = condicao_1.sum()
express_n1 = condicao_2.sum()

semanal_s1 = condicao_3.sum()
semanal_n1 = condicao_4.sum()

estoque_s1 = condicao_5.sum()
estoque_n1 = condicao_6.sum()

programado_s1 = condicao_7.sum()
programado_n1 = condicao_8.sum()

soma_linhas_totais_1 = express_s1 + express_n1 + semanal_s1 + semanal_n1 + estoque_s1 + estoque_n1 + programado_s1 + programado_n1

soma_linhas_express_1 = express_s1 + express_n1
soma_linhas_semanal_1 = semanal_s1 + semanal_n1
soma_linhas_estoque_1 = estoque_s1 + estoque_n1
soma_linhas_programado_1 = programado_s1 + programado_n1

print(f'O total de linhas é {soma_linhas_totais_1} sendo: \n express: {soma_linhas_express_1} \n semanal: {soma_linhas_semanal_1} \n estoque {soma_linhas_estoque_1} \n programado {soma_linhas_programado_1}')

## Separação ##

In [ ]:
# Filtrando a df_doc_2 - separação
df_doc_2 = df_doc_2[df_doc_2['G~Ctg.NF'].isna() | (df_doc_2['G~Ctg.NF'] == '')]
indices_a_remover_4 = df_doc_2[df_doc_2['TpDocVend.'] != 'Z000'].index
df_doc_2 = df_doc_2.drop(indices_a_remover_4)
df_doc_2 = df_doc_2[df_doc_2['Fornmto.'].notna()]

# Transformando colunas de datas
df_doc_2['F~Dt.fatur.'] = pd.to_datetime(df_doc_2['F~Dt.fatur.'])
df_doc_2['A~DtRefClnt.'] = pd.to_datetime(df_doc_2['A~DtRefClnt.'])

linhas = len(df_doc_2)
print(linhas)

## Filtrando apenas o mês de interesse do faturamento ##

In [ ]:
# Filtrando apenas o mês de interesse
data_inicio_faturamento_2 = pd.Timestamp('2024-12-01')
data_fim_faturamento_2 = pd.Timestamp('2024-12-31')
df_doc_2 = df_doc_2[(df_doc_2['F~Dt.fatur.'] >= data_inicio_faturamento) & (df_doc_2['F~Dt.fatur.'] <= data_fim_faturamento)]

linhas_3 = len(df_doc_2)
print(linhas_3)

In [ ]:
# Inserindo coluna com a diferença entre as datas de faturamento e datas do pedido do cliente
df_doc_2['Diferenca_Dias_2'] = (df_doc_2['F~Dt.fatur.'] - df_doc_2['A~DtRefClnt.']).dt.days

# Exibir as primeiras linhas do DataFrame para verificar a nova coluna
print(df_doc_2[['F~Dt.fatur.', 'A~DtRefClnt.', 'Diferenca_Dias_2']].head())

In [ ]:
# Inserindo coluna com o nome do tipo de pedido

# Dicionário para mapear os valores de 'A~MotivoOrd' para os tipos de pedido
mapeamento_tipo_pedido = {
    'Z20': 'Express',
    'Z25': 'Semanal',
    'Z22': 'Estoque',
    'Z35': 'Programado'
}

# Criar a coluna 'Tipo_Pedido' usando o mapeamento
df_doc_2['Tipo_Pedido_2'] = df_doc_2['A~MotivoOrd'].map(mapeamento_tipo_pedido)

df_doc_2.head

In [ ]:
# Inserindo coluna com a verificação se o pedido faturado no mês de interesse atendeu os dias especificados

# Definir as condições de acordo com os valores de 'A~MotivoOrd' e 'Diferenca_Dias'
condicoes_2 = [
    ((df_doc_2['A~MotivoOrd'] == 'Z20') & (df_doc_2['Diferenca_Dias_2'] >= 0) & (df_doc_2['Diferenca_Dias_2'] <= 4)),
    ((df_doc_2['A~MotivoOrd'] == 'Z25') & (df_doc_2['Diferenca_Dias_2'] >= 0) & (df_doc_2['Diferenca_Dias_2'] <= 7)),
    ((df_doc_2['A~MotivoOrd'] == 'Z22') & (df_doc_2['Diferenca_Dias_2'] >= 0) & (df_doc_2['Diferenca_Dias_2'] <= 21)),
    ((df_doc_2['A~MotivoOrd'] == 'Z35') & (df_doc_2['Diferenca_Dias_2'] >= 0) & (df_doc_2['Diferenca_Dias_2'] <= 120))
]

# Definir as escolhas correspondentes a cada condição
escolhas_2 = [
    'Atendeu no Prazo',
    'Atendeu no Prazo',
    'Atendeu no Prazo',
    'Atendeu no Prazo'
]

# Aplicar as condições e escolhas no DataFrame, 'Não Atendeu' é o padrão para quando nenhuma condição é verdadeira
df_doc_2['Atendimento_2'] = np.select(condicoes_2, escolhas_2, default='Não Atendeu')

df_doc_2

In [ ]:
# Contagem de linhas por tipo de pedido - Separação

# Verifica ambas as condições desejadas
condicao_9 = (df_doc_2['Tipo_Pedido_2'] == 'Express') & (df_doc_2['Atendimento_2'] == 'Atendeu no Prazo')
condicao_10 = (df_doc_2['Tipo_Pedido_2'] == 'Express') & (df_doc_2['Atendimento_2'] == 'Não Atendeu')

condicao_11 = (df_doc_2['Tipo_Pedido_2'] == 'Semanal') & (df_doc_2['Atendimento_2'] == 'Atendeu no Prazo')
condicao_12 = (df_doc_2['Tipo_Pedido_2'] == 'Semanal') & (df_doc_2['Atendimento_2'] == 'Não Atendeu')

condicao_13 = (df_doc_2['Tipo_Pedido_2'] == 'Estoque') & (df_doc_2['Atendimento_2'] == 'Atendeu no Prazo')
condicao_14 = (df_doc_2['Tipo_Pedido_2'] == 'Estoque') & (df_doc_2['Atendimento_2'] == 'Não Atendeu')

condicao_15 = (df_doc_2['Tipo_Pedido_2'] == 'Programado') & (df_doc_2['Atendimento_2'] == 'Atendeu no Prazo')
condicao_16 = (df_doc_2['Tipo_Pedido_2'] == 'Programado') & (df_doc_2['Atendimento_2'] == 'Não Atendeu')

# Contar as linhas que satisfazem a condição
express_s2 = condicao_9.sum()
express_n2 = condicao_10.sum()

semanal_s2 = condicao_11.sum()
semanal_n2 = condicao_12.sum()

estoque_s2 = condicao_13.sum()
estoque_n2 = condicao_14.sum()

programado_s2 = condicao_15.sum()
programado_n2 = condicao_16.sum()

soma_linhas_totais_2 = express_s2 + express_n2 + semanal_s2 + semanal_n2 + estoque_s2 + estoque_n2 + programado_s2 + programado_n2

soma_linhas_express_2 = express_s2 + express_n2
soma_linhas_semanal_2 = semanal_s2 + semanal_n2
soma_linhas_estoque_2 = estoque_s2 + estoque_n2
soma_linhas_programado_2 = programado_s2 + programado_n2

print(f'O total de linhas é {soma_linhas_totais_2} sendo: \n express: {soma_linhas_express_2} \n semanal: {soma_linhas_semanal_2} \n estoque {soma_linhas_estoque_2} \n programado {soma_linhas_programado_2}')

## Pendente atendimento - ZVA05 ##

* Lembrar de SEMPRE ajustar as colunas "Análise", "Análise 2" e "Classif. Ped. Rede"
* Apagar primeira linha da planilha para identificar as colunas

In [ ]:
# Criando coluna com a data prevista de atendimento 
df_05['A~Em'] = pd.to_datetime(df_05['A~Em'])

def calculo_data_limite(row):
    if row['Classif. Ped. Rede'] == 'Express':
        return row['A~Em'] + pd.Timedelta(days=4)
    elif row['Classif. Ped. Rede'] == 'Semanal':
        return row['A~Em'] + pd.Timedelta(days=7)
    elif row['Classif. Ped. Rede'] == 'Estoque':
        return row['A~Em'] + pd.Timedelta(days=21)
    elif row['Classif. Ped. Rede'] == 'Programado':
        return row['A~Em'] + pd.Timedelta(days=120)
    else:
        return row['A~Em']  

# Aplicando a função para criar a nova coluna
df_05['data limite atendimento'] = df_05.apply(calculo_data_limite, axis=1)

df_05.dtypes


## Filtrando apenas o mês de interesse ##


In [ ]:
data_inicio_prevista = pd.Timestamp('2024-12-01')
data_fim_prevista = pd.Timestamp('2024-12-31')
df_05 = df_05[(df_05['data limite atendimento'] >= data_inicio_faturamento) & (df_05['data limite atendimento'] <= data_fim_faturamento)]

linhas_4 = len(df_05)
print(linhas_4)

In [ ]:
# Contagem de linhas por tipo de pedido - Pendente Atendimento

# Verifica ambas as condições desejadas
condicao_17 = (df_05['Classif. Ped. Rede'] == 'Express') & (df_05['Análise 2'] == 'Disponível')
condicao_18 = (df_05['Classif. Ped. Rede'] == 'Express') & (df_05['Análise 2'] == 'Pendente Atendimento')

condicao_19 = (df_05['Classif. Ped. Rede'] == 'Semanal') & (df_05['Análise 2'] == 'Disponível')
condicao_20 = (df_05['Classif. Ped. Rede'] == 'Semanal') & (df_05['Análise 2'] == 'Pendente Atendimento')

condicao_21 = (df_05['Classif. Ped. Rede'] == 'Estoque') & (df_05['Análise 2'] == 'Disponível')
condicao_22 = (df_05['Classif. Ped. Rede'] == 'Estoque') & (df_05['Análise 2'] == 'Pendente Atendimento')

condicao_23 = (df_05['Classif. Ped. Rede'] == 'Programado') & (df_05['Análise 2'] == 'Disponível')
condicao_24 = (df_05['Classif. Ped. Rede'] == 'Programado') & (df_05['Análise 2'] == 'Pendente Atendimento')

# Contar as linhas que satisfazem a condição
express_s3 = condicao_17.sum()
express_n3 = condicao_18.sum()

semanal_s3 = condicao_19.sum()
semanal_n3 = condicao_20.sum()

estoque_s3 = condicao_21.sum()
estoque_n3 = condicao_22.sum()

programado_s3 = condicao_23.sum()
programado_n3 = condicao_24.sum()

soma_linhas_totais_3 = express_s3 + express_n3 + semanal_s3 + semanal_n3 + estoque_s3 + estoque_n3 + programado_s3 + programado_n3

soma_linhas_express_3 = express_s3 + express_n3
soma_linhas_semanal_3 = semanal_s3 + semanal_n3
soma_linhas_estoque_3 = estoque_s3 + estoque_n3
soma_linhas_programado_3 = programado_s3 + programado_n3

print(f'O total de linhas é {soma_linhas_totais_3} sendo: \n express: {soma_linhas_express_3} \n semanal: {soma_linhas_semanal_3} \n estoque {soma_linhas_estoque_3} \n programado {soma_linhas_programado_3}')

## TAXA DE ATENDIMENTO 

In [ ]:
# Taxa Global
taxa_global = round(((express_s1 + express_s2 + express_s3 + semanal_s1 + semanal_s2 + semanal_s3 + estoque_s1 + estoque_s2 + estoque_s3 + programado_s1 + programado_s2 + programado_s3)/(soma_linhas_totais_1 + soma_linhas_totais_2 + soma_linhas_totais_3)*100), 2)
print(f'Taxa Global = {taxa_global}%')

# Taxa Express
taxa_ex = round(((express_s1 + express_s2 + express_s3)/(express_s1 + express_s2 + express_s3 + express_n1 + express_n2 + express_n3))*100, 2)
print(f'Taxa Express = {taxa_ex}%')

# Taxa Semanal
taxa_sem = round(((semanal_s1 + semanal_s2 + semanal_s3) / (semanal_s1 + semanal_s2 + semanal_s3 + semanal_n1 + semanal_n2 + semanal_n3))*100, 2)
print(f'Taxa Semanal = {taxa_sem}%')

# Taxa Estoque
taxa_est = round(((estoque_s1 + estoque_s2 + estoque_s3)/(estoque_s1 + estoque_s2 + estoque_s3 + estoque_n1 + estoque_n2 + estoque_n3))*100, 2)
print(f'Taxa Estoque = {taxa_est}%')

# Taxa Programado
taxa_pro = round(((programado_s1 + programado_s2 + programado_s3)/(programado_s1 + programado_s2 + programado_s3 + programado_n1 + programado_n2 + programado_n3))*100, 2)
print(f'Taxa Programado = {taxa_pro}%')